# Example to_hdf calls

Initialize the simulation with `the tardis_example.yml` configuration file.

In [2]:
from tardis.io.config_reader import Configuration
from tardis.model import Radial1DModel
from tardis.simulation import Simulation

# Must have the tardis_example folder in the working directory.
config_fname = 'tardis_example/tardis_example.yml'
tardis_config = Configuration.from_yaml(config_fname)

model = Radial1DModel(tardis_config)
simulation = Simulation(tardis_config)

tardis.io.config_reader - INFO - Reading Atomic Data from tardis_example/kurucz_cd23_chianti_H_He.h5
tardis.atomic - INFO - Read Atom Data with UUID=5ca3035ca8b311e3bb684437e69d75d7 and MD5=21095dd25faa1683f4c90c911a00c3f8
tardis.io.config_reader - INFO - "initial_t_inner" is not specified in the plasma section - initializing to 9933.95199592 K with given luminosity
tardis.io.config_reader - WARNING - No "species" given - ignoring other NLTE options given:
OrderedDict([('coronal_approximation', False), ('classical_nebular', False)])
tardis.model - INFO - Calculating J_blues for radiative_rates_type=dilute-blackbody


Run the simulation while storing all its iterations to an HDF file.

The first parameter is the path where the HDF file should be stored. The second parameter determines which properties will be stored. When its value is 'input', only Input plasma properties will be stored. The third parameter, hdf_last_only, if True will only store the last iteration of the simulation, otherwise every iteration will be stored.

In [3]:
simulation.legacy_run_simulation(model, '/tmp/full_example.hdf', 'full', hdf_last_only=False)

tardis.simulation.base - INFO - Remaining run 20
tardis.simulation.base - INFO - Luminosity emitted = 7.96033e+42 erg / s Luminosity absorbed = 2.64155e+42 erg / s Luminosity requested = 1.05928e+43 erg / s
tardis.simulation.base - INFO - Plasma stratification:
	             t_rad    next_t_rad         w    next_w
	Shell                                               
	0      9926.501965  10178.847299  0.400392  0.497507
	5      9852.611678  10369.295730  0.211205  0.187775
	10     9779.813302  10216.758348  0.142695  0.114941
	15     9708.082813   9970.054217  0.104556  0.084019


tardis.simulation.base - INFO - t_inner 9933.952 K -- next t_inner 11459.378 K
tardis.model - INFO - Calculating J_blues for radiative_rates_type=dilute-blackbody
py.warnings - WARNING - /home/ftsamis/anaconda2/envs/tardis/lib/python2.7/site-packages/pandas/io/pytables.py:2577: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferre

Open the stored HDF file with pandas and print its structure.

In [6]:
import pandas as pd
data = pd.HDFStore('/tmp/full_example.hdf')
print data

<class 'pandas.io.pytables.HDFStore'>
File path: /tmp/full_example.hdf
/simulation1/model/arrays                                            frame        (shape->[20,4])    
/simulation1/model/metadata                                          series       (shape->[1])       
/simulation1/model/plasma/abundance                                  frame        (shape->[6,20])    
/simulation1/model/plasma/atomic_mass                                series       (shape->[6])       
/simulation1/model/plasma/beta_rad                                   series       (shape->[20])      
/simulation1/model/plasma/beta_sobolev                               frame        (shape->[29224,20])
/simulation1/model/plasma/density                                    series       (shape->[20])      
/simulation1/model/plasma/electron_densities                         series       (shape->[20])      
/simulation1/model/plasma/excitation_energy                          series       (shape->[4439])    
/simulation

Access `model.plasma.density` of the 9th simulation, which is a one-dimensional array

In [7]:
print data['/simulation9/model/plasma/density']

0     7.542804e-14
1     5.728475e-14
2     4.396074e-14
3     3.406287e-14
4     2.663135e-14
5     2.099596e-14
6     1.668287e-14
7     1.335311e-14
8     1.076154e-14
9     8.729085e-15
10    7.123652e-15
11    5.846921e-15
12    4.825093e-15
13    4.002324e-15
14    3.336039e-15
15    2.793540e-15
16    2.349550e-15
17    1.984397e-15
18    1.682677e-15
19    1.432260e-15
dtype: float64


Scalars are stored in a `scalars` `pandas.Series` for every module. For example to access `model.t_inner` of the 9th iteration of the simulation, one would need to do the following.

Note: Quantities are always stored as their SI values.

In [9]:
print data['/simulation9/model/scalars']['t_inner']

11257.7713721


# Breakdown of the various to_hdf methods
Every module in TARDIS has its own `to_hdf` method responsible to store its own data to an HDF file.

### Plasma
The following call will store every plasma property to `/tmp/plasma_output.hdf` under `/parent/plasma`

In [3]:
model.plasma.to_hdf('/tmp/plasma_output.hdf', path='parent')

py.warnings - WARNING - /home/ftsamis/anaconda2/envs/tardis/lib/python2.7/site-packages/pandas/io/pytables.py:2577: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block0_values] [items->[0]]

  warnings.warn(ws, PerformanceWarning)



In [4]:
import pandas
with pandas.HDFStore('/tmp/plasma_output.hdf') as data:
    print data

<class 'pandas.io.pytables.HDFStore'>
File path: /tmp/plasma_output.hdf
/parent/plasma/abundance                                 frame        (shape->[6,20])    
/parent/plasma/atomic_mass                               series       (shape->[6])       
/parent/plasma/beta_rad                                  series       (shape->[20])      
/parent/plasma/beta_sobolev                              frame        (shape->[29224,20])
/parent/plasma/density                                   series       (shape->[20])      
/parent/plasma/electron_densities                        series       (shape->[20])      
/parent/plasma/excitation_energy                         series       (shape->[4439])    
/parent/plasma/f_lu                                      series       (shape->[29224])   
/parent/plasma/g                                         series       (shape->[4439])    
/parent/plasma/g_electron                                series       (shape->[20])      
/parent/plasma/general_level

Plasma's `to_hdf` method can also accept a `collection` parameter which can specify which types of plasma properties will be stored. For example if we wanted to only store Input plasma properties, we would do the following:

In [11]:
from tardis.plasma.properties.base import Input
model.plasma.to_hdf('/tmp/plasma_input_output.hdf', collection=[Input])

In [12]:
import pandas
with pandas.HDFStore('/tmp/plasma_input_output.hdf') as data:
    print data

<class 'pandas.io.pytables.HDFStore'>
File path: /tmp/plasma_input_output.hdf
/plasma/abundance            frame        (shape->[6,20])    
/plasma/density              series       (shape->[20])      
/plasma/j_blues              frame        (shape->[29224,20])
/plasma/scalars              series       (shape->[2])       
/plasma/t_rad                series       (shape->[20])      
/plasma/w                    series       (shape->[20])      


### Model
The following call will store properties of the `Radial1DModel` to `/tmp/model_output.hdf` under `/model`. Additionally, it will automatically call `model.plasma.to_hdf`, since plasma is also a property of the model.

In [ ]:
model.to_hdf('/tmp/model_output.hdf')

### MontecarloRunner
The following call will store properties of the `MontecarloRunner` to `/tmp/runner_output.hdf` under `/runner`

In [7]:
simulation.runner.to_hdf('/tmp/runner_output.hdf')

In [8]:
import pandas
with pandas.HDFStore('/tmp/runner_output.hdf') as data:
    print data

<class 'pandas.io.pytables.HDFStore'>
File path: /tmp/runner_output.hdf
/runner/j_estimator                 series       (shape->[20])    
/runner/nu_bar_estimator            series       (shape->[20])    
/runner/output_energy               series       (shape->[100000])
/runner/output_nu                   series       (shape->[100000])
